In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from py.is_rebalance import is_rebalance
from py.signals import signals
from py.split_df import split_df
from py.simulate import simulate

In [2]:
# VARIABLES

# We want 180 day windows (6 months) with 120 day overlap (4 months)
# Since our dataframe is in hours, multiply by 24
window_len = 24 * 180
overlap = 24 * 120

# Assets traded
assets = ['ETH', 'USD']

# Moving average intervals used
moving_averages = [50, 100, 200]

# Potential ETH to DAI allocations from bullish signals
bull_allocation = [
    [0.85, 0.15],
    [0.80, 0.20],
    [0.75, 0.25],
    [0.70, 0.30],
    [0.65, 0.35]
]

# List of allocations used, with the inverse allocation for bearish signals
allocation_lst = [{'bull': b,
                   'neutral': [0.50, 0.50],
                   'bear': b[::-1]}
                  for b in bull_allocation]


# Minimum difference in weighting needed to rebalance without a new signal
# This prevents unnecessary rebalancing
wiggle_room_lst = np.arange(0, 0.21, 0.02)

In [3]:
df = pd.read_csv('../data/ETH-USDT.csv', usecols=['date', 'close']).rename({'close':'ETH'}, axis=1)
df['date'] = pd.to_datetime(df['date'])
df['USD'] = 1

# Create columns to 
df['rebalance'] = is_rebalance(df['date'])
df['signal'] = signals(df['ETH'], df['rebalance'], *moving_averages)

# Split dataframe into windows 
dfs = split_df(df.to_dict(orient='records'), overlap, window_len)

results = []

for allocation in allocation_lst:
    for wiggle_room in wiggle_room_lst:
        result = {
            'wiggle_room': wiggle_room,
            'allocation': '/'.join(str(x) for x in allocation['bull']),
        }

        # Add result for each split dataframe
        for df_split in dfs:
            start = datetime.strftime(df_split[0]['date'], '%Y.%m.%d')
            end = datetime.strftime(df_split[-1]['date'], '%Y.%m.%d')

            _, _, performance = simulate(assets, allocation, wiggle_room, df_split)

            result[start + '-' + end] = performance

        # Save result to results
        results.append(result)


# Convert dict to dataframe
df_results = pd.DataFrame.from_records(results)
df_results['sum'] = df_results.drop(['wiggle_room', 'allocation'], axis=1).sum(axis=1)

# Sort
df_results = df_results.sort_values('sum', ascending=False)


# Save signals and performance to CSV
df.to_csv('backtests/signals.csv', index=False)
df_results.to_csv('backtests/performance.csv', index=False)

In [5]:
df_results.head()

,wiggle_room,allocation,2017.08.16 - 2018.01.14,2017.10.06 - 2018.03.06,2017.11.25 - 2018.04.25,2018.01.14 - 2018.06.14,2018.03.06 - 2018.08.04,2018.04.25 - 2018.09.23,2018.06.14 - 2018.11.12,2018.08.04 - 2019.01.01,2018.09.23 - 2019.02.20,2018.11.12 - 2019.04.11,2019.01.01 - 2019.06.01,2019.02.20 - 2019.07.20,2019.04.11 - 2019.09.09,2019.05.31 - 2019.10.29,2019.07.20 - 2019.12.18,2019.09.09 - 2020.02.06,2019.10.29 - 2020.03.27,sum
6,0.12,0.85/0.15,-425.965628,-712.444817,-250.969542,2101.998860,1540.164528,1141.851670,1426.445933,2686.738376,1635.427605,1726.818947,-214.339982,-424.842728,-185.993311,725.554161,419.701239,213.352388,449.856898,11853.354598
4,0.08,0.85/0.15,-447.799850,-748.571175,-282.241089,2207.850375,1531.614109,1112.034530,1378.034851,2709.780372,1623.716844,1622.842195,-250.069083,-391.978458,-103.720199,735.148453,360.348314,235.043474,484.430384,11776.464049
7,0.14,0.85/0.15,-425.965628,-755.373545,-251.694753,1925.533989,1423.825974,1178.989852,1525.414125,2840.145167,1675.927111,1731.554799,-211.997432,-396.381790,-222.399319,661.743908,419.701239,174.752461,435.388584,11729.164740
5,0.10,0.85/0.15,-449.382185,-759.589505,-250.969542,2101.998860,1540.164528,1132.284117,1402.412106,2618.406274,1635.427605,1622.842195,-257.640009,-425.228868,-134.228928,742.372129,414.337212,223.714430,449.856898,11606.777318
0,0.00,0.85/0.15,-475.184920,-739.096048,-258.341920,2108.315278,1581.133443,1061.546668,1340.645452,2737.178005,1576.074597,1636.227791,-274.806148,-385.057091,-111.612861,724.171199,355.854287,163.472716,491.674792,11532.195239
